## Конвертация XLSX в Parquet

### Версия Python

In [1]:
!python -V

Python 3.12.6


### Импорт необходимых библиотек

In [2]:
# Подавление предупреждений
import warnings
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

import os
import time
import polars as pl
import fastexcel
import multiprocessing
import tqdm as tq
import jupyterlab as jlab
import ipywidgets

from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

### Версии необходимых библиотек

In [3]:
packages = [
    "Polars", "Fastexcel", "Ipywidgets", "Tqdm", "JupyterLab"
]

package_objects = [
    pl, fastexcel, ipywidgets, tq, jlab
]

versions = list(map(lambda obj: obj.__version__, package_objects))

columns_order = ["№", "Библиотека", "Версия"]
df_pkgs = (
    pl.DataFrame({
        columns_order[1]: packages,
        columns_order[2]: versions
    })
    .with_columns(pl.arange(1, pl.lit(len(packages)) + 1).alias(columns_order[0]))
    .select(columns_order)
)

display(df_pkgs)

path2reqs = "."
reqs_name = "requirements.txt"

def get_packages_and_versions():
    """Генерация строк с библиотеками и их версиями в формате: библиотека==версия"""
    
    for package, version in zip(packages, versions):
        yield f"{package.lower()}=={version}\n"

with open(os.path.join(path2reqs, reqs_name), "w", encoding = "utf-8") as f:
    f.writelines(get_packages_and_versions())

№,Библиотека,Версия
i64,str,str
1,"""Polars""","""1.9.0"""
2,"""Fastexcel""","""0.11.6"""
3,"""Ipywidgets""","""8.1.5"""
4,"""Tqdm""","""4.66.5"""
5,"""JupyterLab""","""4.2.5"""


### Чтение всех `xlsx` файлов

In [5]:
start_time = time.time()

# Получение количества ядер процессора
num_cores_cpu = multiprocessing.cpu_count()

# Определение пути к папке с xlsx файлами
path2files = Path("/Users/dl/GitHub/RecSysApp/data/ПУДы")

# Получение списка всех xlsx файлов
xlsx_files = list(path2files.rglob("*.xlsx"))

def load_xlsx(file_path):
    """Чтение xlsx файла"""
    
    return pl.read_excel(source = file_path)

# Ограничение количества потоков
max_workers = min(num_cores_cpu, len(xlsx_files))

# Чтение файлов параллельно с использованием ThreadPoolExecutor
with ThreadPoolExecutor(max_workers = max_workers) as executor:
    dfs = list(tqdm(executor.map(load_xlsx, xlsx_files), desc = "Обработка xlsx файлов", total = len(xlsx_files), unit = "файл"))

# Объединение всех файлов в один DataFrame
df = pl.concat(dfs)

print(f"Количество строк: {df.shape[0]}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Время выполнения: {elapsed_time:.6f} секунд")

Обработка xlsx файлов: 100%|██████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 89.10файл/s]

Количество строк: 52211
Время выполнения: 4.634208 секунд


In [6]:
df[0]["Русскоязычное название дисциплины"][0]

'Выставочная деятельность в сфере искусства и культуры'

In [7]:
# Путь для сохранения
save_path = Path("/Users/dl/GitHub/RecSysApp/data")

def prepare_save_path(save_path, rmtree = False):
    """Создание/очистка целевой папки"""
    
    if save_path.exists() and rmtree:
        shutil.rmtree(save_path)
    save_path.mkdir(parents = True, exist_ok = True)

prepare_save_path(save_path)

def save2parquet(df, file_path):
    """Cохранения DataFrame в Parquet"""
    
    df.write_parquet(file_path)

# Сохранение выборок
save2parquet(df, save_path / "ПУДы.parquet")

### Тестирование Parquet файла

In [81]:
start_time = time.time()

# Получение количества ядер процессора
num_cores_cpu = multiprocessing.cpu_count()

# Определение пути к папке с Parquet файлами
path2files = Path("/Users/dl/GitHub/RecSysApp/data")

# Получение списка всех Parquet файлов
parquet_files = list(path2files.rglob("*ПУДы*.parquet"))

def load_parquet(file_path):
    """Чтение Parquet файла"""
    
    return pl.read_parquet(file_path)

# Ограничение количества потоков
max_workers = min(num_cores_cpu, len(parquet_files))

# Чтение Parquet файлов
with ThreadPoolExecutor(max_workers = max_workers) as executor:
    dfs = list(tqdm(executor.map(
        load_parquet, parquet_files), desc = "Чтение Parquet файлов", total = len(parquet_files), unit = "файл")
    )

# Объединение всех файлов в один DataFrame
df = pl.concat(dfs)

print(f"Количество строк: {df.shape[0]}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Время выполнения: {elapsed_time:.6f} секунд")

Чтение Parquet файлов: 100%|██████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.74файл/s]

Количество строк: 52211
Время выполнения: 0.584225 секунд


In [82]:
df[0]["Русскоязычное название дисциплины"][0]

'Выставочная деятельность в сфере искусства и культуры'